In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Set image dimensions and other parameters
image_size = (224, 224)  # ResNet50 expects 224x224 images
batch_size = 32
epochs = 50

# Define the paths to the dataset (adjust these paths as needed)
base_dir = './data'  # Root directory where the 'Angry', 'Happy', 'Sad', 'Fear' folders are located

# 1. **Data Augmentation**: Apply common transformations to prevent overfitting
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # This will split off 20% for validation
)

# Validation and Test generators (only rescaling)
val_test_datagen = ImageDataGenerator(rescale=1./255)

# 2. **Load the data into train and validation sets using validation_split**
train_data_gen = train_datagen.flow_from_directory(
    base_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Assuming multi-class classification (4 classes)
    shuffle=True,
    subset='training'  # This is the training data subset (80%)
)

val_data_gen = train_datagen.flow_from_directory(
    base_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Multi-class classification
    subset='validation'  # This is the validation data subset (20%)
)

# 3. **Define the model with ResNet50 as the base model**
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the ResNet50 model so we only train the top layers
base_model.trainable = False

# Create the model by adding custom layers on top of ResNet50
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),  # Dropout layer to prevent overfitting
    layers.Dense(4, activation='softmax')  # Output layer with 4 categories
])

# 4. **Compile the model**
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),  # A low learning rate for fine-tuning
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 5. **Setup callbacks for early stopping and model checkpointing**
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
  
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)


In [ ]:
import splitfolders
splitfolders.ratio("./dataset", output="./data-split", seed=1337, ratio=(0.8, 0.1,0.1), group_prefix=None, move=False)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    './dataset-split/train/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'  # Ensure this is 'categorical' for multi-class classification
)

valid_generator = valid_datagen.flow_from_directory(
    './dataset-split/val/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'  # Ensure this is 'categorical' for multi-class classification
)


Found 482 images belonging to 4 classes.
Found 74 images belonging to 4 classes.
Found 138 images belonging to 4 classes.


In [5]:
# ResNet50 model without the top layer (for transfer learning)
resnet_50 = ResNet50(
    include_top=False,     # Exclude the fully connected layers
    weights='imagenet',    # Load pre-trained ImageNet weights
    input_shape=(224, 224, 3)  # Input image shape
)

# Freeze all layers of ResNet50 initially
for layer in resnet_50.layers[-4:]:  # Unfreeze the last 4 layers of ResNet50
    layer.trainable = True

# Add custom layers on top of ResNet50
x = resnet_50.output
x = GlobalAveragePooling2D()(x)  # Global Average Pooling
x = BatchNormalization()(x)      # Batch Normalization
x = Dense(128, activation='relu')(x)  # Fully connected layer with 128 units
x = Dropout(0.5)(x)              # Dropout layer to prevent overfitting

# Output layer (softmax for 4 classes)
predictions = Dense(4, activation='softmax')(x)

# Final model
model = Model(inputs=resnet_50.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-5), loss="categorical_crossentropy", metrics=["accuracy"])


In [6]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Function to train the model
def trainModel(model, train_generator, valid_generator, epochs, class_weights=None):
    batch_size = 32
    
    # Callbacks to stop training early and reduce learning rate
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
    
    # Train the model with class weights if provided
    return model.fit(
        train_generator,
        validation_data=valid_generator,
        epochs=epochs,
        batch_size=batch_size,
        class_weight=class_weights,  # Include class weights
        callbacks=[early_stopping, reduce_lr]
    )

# Define class weights (adjust based on your dataset)
class_weights = {0: 1., 1: 1., 2: 2., 3: 2.}

# Call the trainModel function
model_history = trainModel(
    model=model, 
    train_generator=train_generator, 
    valid_generator=valid_generator, 
    epochs=50, 
    class_weights=class_weights
)


c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
TypeError: `generator` yielded an element that did not match the expected structure. The expected structure was (tf.float32, tf.float32, tf.float32), but the yielded element was (array([[[[ -90.939    ,  -94.779    ,  -97.68     ],
         [ -89.98149  ,  -93.82149  ,  -96.72249  ],
         [ -88.99046  ,  -92.83046  ,  -95.73146  ],
         ...,
         [ -31.971985 ,  -18.192192 ,   -9.299789 ],
         [ -38.334053 ,  -25.563263 ,  -16.464264 ],
         [ -49.65444  ,  -37.47946  ,  -28.380463 ]],

        [[ -90.939    ,  -94.779    ,  -97.68     ],
         [ -90.405525 ,  -94.24553  ,  -97.14653  ],
         [ -89.414505 ,  -93.2545   ,  -96.1555   ],
         ...,
         [ -40.45012  ,  -27.790703 ,  -18.691704 ],
         [ -50.228188 ,  -38.26097  ,  -29.161972 ],
         [ -52.015617 ,  -41.24002  ,  -32.14102  ]],

        [[ -90.939  

In [23]:
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print(f"Test Accuracy: {test_acc}")
print(f"Test Loss: {test_loss}")


3/3 - 4s - 1s/step - accuracy: 0.5541 - loss: 1.3819
Test Accuracy: 0.5540540814399719
Test Loss: 1.3819315433502197
